In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import requests
import time
import hmac
import hashlib
import base64

# Signature class for generating API signature
class Signature:
    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
        return base64.b64encode(hash.digest()).decode("utf-8")

# Function to get the header for the API request
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = Signature.generate(timestamp, method, uri, secret_key)
    return {
        'Content-Type': 'application/json; charset=UTF-8',
        'X-Timestamp': timestamp,
        'X-API-KEY': api_key,
        'X-Customer': str(customer_id),
        'X-Signature': signature
    }

# Function to get the results of related keywords from the Naver API
def get_results(hintKeywords):
    BASE_URL = 'https://api.naver.com'
    API_KEY = '010000000013d50306951198c56725e6d082916ed4870b354963d7e7119eb766144d8c58c0'  # Replace with your actual API_KEY
    SECRET_KEY = 'AQAAAAAT1QMGlRGYxWcl5tCCkW7UFW5GoTbNMAkwsweVpmyjWw=='  # Replace with your actual SECRET_KEY
    CUSTOMER_ID = '2496501'  # Replace with your actual CUSTOMER_ID

    uri = '/keywordstool'
    method = 'GET'

    params = {
        'hintKeywords': hintKeywords,
        'showDetail': '1'
    }

    r = requests.get(BASE_URL + uri, params=params,
                     headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

    if r.status_code == 200:
        return pd.DataFrame(r.json()['keywordList'])
    else:
        messagebox.showerror("Error", f"API Error: {r.status_code}, {r.text}")
        return None

# Function to update the results listbox with related keywords
def update_related_keywords():
    hintKeywords = keyword_entry.get()  # Get the input from the keyword entry
    if not hintKeywords:
        messagebox.showwarning("Input Error", "Please enter a keyword.")
        return
    results_df = get_results(hintKeywords)
    
    if results_df is not None:
        # Check the column names in the API response
        print("API Response Columns:", results_df.columns)
        
        # Now using 'relKeyword' as the correct column name
        if 'relKeyword' in results_df.columns:
            related_keywords_listbox.delete(0, tk.END)  # Clear previous results
            for keyword in results_df['relKeyword']:
                related_keywords_listbox.insert(tk.END, keyword)
        else:
            messagebox.showerror("Error", "API response does not contain the expected 'relKeyword' column.")



# Function to save the excluded words
def save_excluded_words():
    excluded_words = exclude_entry.get()  # Get the input from the exclude entry
    if excluded_words:
        with open("excluded_words.txt", "a") as file:
            file.write(excluded_words + "\n")
        exclude_entry.delete(0, tk.END)  # Clear the exclude entry
        messagebox.showinfo("Saved", "Excluded words have been saved.")

# Function to update detailed keywords based on input
def update_detailed_keywords():
    search_term = detailed_keywords_entry.get()  # Get the input from the entry field
    
    if not search_term:
        messagebox.showwarning("Input Error", "Please enter a term to search.")
        return
    
    # Iterate over the related keywords list to find matches
    detailed_keywords_listbox.delete(0, tk.END)  # Clear previous results
    
    for keyword in related_keywords_listbox.get(0, tk.END):  # Iterate through all related keywords
        if search_term in keyword:
            detailed_keywords_listbox.insert(tk.END, keyword)  # Add matching keywords to the listbox
    
    if detailed_keywords_listbox.size() == 0:
        messagebox.showinfo("No Match", f"No keywords contain '{search_term}'.")


# Function to update the main keywords listbox
def update_main_keywords():
    main_keywords = main_keywords_entry.get()  # Get the input from the main keywords entry
    if main_keywords:
        main_keywords_listbox.insert(tk.END, main_keywords)
        main_keywords_entry.delete(0, tk.END)  # Clear the entry

# Creating the main window
root = tk.Tk()
root.title("Keyword Tool")
root.geometry("600x800")

# Create and place widgets

# Keyword input section
keyword_label = tk.Label(root, text="Enter Keyword:")
keyword_label.pack(pady=5)
keyword_entry = tk.Entry(root, width=50)
keyword_entry.pack(pady=5)
search_button = tk.Button(root, text="Search Related Keywords", command=update_related_keywords)
search_button.pack(pady=5)

# Related keywords section
related_keywords_label = tk.Label(root, text="Related Keywords:")
related_keywords_label.pack(pady=5)
related_keywords_listbox = tk.Listbox(root, width=50, height=10)
related_keywords_listbox.pack(pady=5)

# Exclude words section
exclude_label = tk.Label(root, text="Enter Excluded Words:")
exclude_label.pack(pady=5)
exclude_entry = tk.Entry(root, width=50)
exclude_entry.pack(pady=5)
save_exclude_button = tk.Button(root, text="Save Excluded Words", command=save_excluded_words)
save_exclude_button.pack(pady=5)

# Detailed keywords section
detailed_keywords_label = tk.Label(root, text="Detailed Keywords:")
detailed_keywords_label.pack(pady=5)
detailed_keywords_listbox = tk.Listbox(root, width=50, height=10)
detailed_keywords_listbox.pack(pady=5)
detailed_keywords_entry = tk.Entry(root, width=50)
detailed_keywords_entry.pack(pady=5)
add_detailed_keywords_button = tk.Button(root, text="세키워드 추가", command=update_detailed_keywords)
add_detailed_keywords_button.pack(pady=5)

# Main keywords section
main_keywords_label = tk.Label(root, text="Main Keywords:")
main_keywords_label.pack(pady=5)
main_keywords_listbox = tk.Listbox(root, width=50, height=10)
main_keywords_listbox.pack(pady=5)
main_keywords_entry = tk.Entry(root, width=50)
main_keywords_entry.pack(pady=5)
add_main_keywords_button = tk.Button(root, text="Add Main Keywords", command=update_main_keywords)
add_main_keywords_button.pack(pady=5)

print("테스트 1")

# Run the Tkinter event loop
root.mainloop()





테스트 1
API Response Columns: Index(['relKeyword', 'monthlyPcQcCnt', 'monthlyMobileQcCnt',
       'monthlyAvePcClkCnt', 'monthlyAveMobileClkCnt', 'monthlyAvePcCtr',
       'monthlyAveMobileCtr', 'plAvgDepth', 'compIdx'],
      dtype='object')
